# Variational multi-scale finite element solution for compressible <b> 
# Navier-Stokes equations - Symbolic Differentiation Formulation

## <center>Elisa Magliozzi<center> 

### <center> *November 2017*<center>


The Variational Multiscale formulation for the compressible Navier-Stokes equations presented in this document was developed by the Dr.-Ing. Camilo A. Bayona Roa as part of his PhD thesis on $\textit{"Adaptive Mesh Simulations of Compressible Flows using Stabilized Formulations"}$, under the supervision of the Professor Ramon Codina and the Dr.-Ing. Joan Baiges.

## List of Symbols

In [47]:
from KratosMultiphysics import *

from sympy import *
from sympy_fe_utilities import *
import pprint

params = {
            "dim": 3,			                                    # Dimension
            "mu": Symbol('mu', positive = True),			        # Dynamic viscosity 
            "h" : Symbol('h', positive = True),	        	        # Element size
            "lambda" : Symbol('lambda', positive = True),           # Thermal Conductivity of the fluid
            "c_v" : Symbol('c_v', positive = True),			        # Specific Heat at Constant volume
            "gamma": Symbol('gamma',positive = True),			    # Gamma (Cp/Cv) 
            "stab_c1" : Symbol('stab_c1', positive = True),			# Algorithm constant
            "stab_c2" : Symbol('stab_c2', positive = True),			# Algorithm constant
        
        }

dim = params["dim"]                         # Define Dimension in params.py  

BlockSize = dim+2 					        # Dimension of the vector of Unknowns
do_simplifications = False
mode = "c"                                  # Output mode to a c++ file

if(dim == 2):
    nnodes = 3
elif(dim == 3):
    nnodes = 4

impose_partion_of_unity = False
N,DN = DefineShapeFunctions(nnodes, dim, impose_partion_of_unity)

# Unknown fields definition (Used later for the gauss point interpolation)
U = DefineMatrix('U',nnodes,BlockSize)	     # Vector of Unknowns ( Density,Velocity[dim],Total Energy )
Un = DefineMatrix('Un',nnodes,BlockSize)     # Vector of Unknowns one step back
Unn = DefineMatrix('Unn',nnodes,BlockSize)   # Vector of Unknowns two steps back
r = DefineVector('r',nnodes)                 # Sink term    #COMMENT for manufactured solution

# Test functions defintiion
w = DefineMatrix('w',nnodes,BlockSize)	    # Variables field test

# External terms definition
f_ext = DefineMatrix('f_ext',nnodes,dim)    # Forcing term #COMMENT for manufactured solution

# Definition of other symbols
bdf0 = Symbol('bdf0')                       # Backward differantiation coefficients
bdf1 = Symbol('bdf1')
bdf2 = Symbol('bdf2')
v_sc = Symbol('v_sc')                       # Shock capturing Viscosity
k_sc = Symbol('k_sc')                       # Shock capturing Conductivity


## 1. Compressible Navier-Stokes formulation

The physics related to a compressible fluid flow can fully described by the compressible Navier-Stokes.
It is necessary to consider the comprimibility effects of a fluid whenever the Mach number (ratio between the velocity and the local speed of sound) is greater than 0.3.
The solution implemented is based on the finite element method for fluids, using a variational multiscale framework to approximate the unresolved subscales.
A newtonian viscous fluid is considered.

1. Conservation of mass equation

    \begin{equation}
    \label{MassConserv}
    \frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \textbf{u})  = 0.
    \end{equation}

2. Conservation of momentum equation

    \begin{equation}
    \label{MomentumCons}
    \frac{\partial (\rho \mathbf{u})}{\partial t} + \nabla \big(\rho \mathbf{u}\big) \mathbf{u} +\nabla \big(p\mathbf{I}-\mathbf{\tau}\big) = \rho \mathbf{f}
    \end{equation}
    
3. Conservation of energy equation

    \begin{equation}
    \label{EnergyCons}
    \frac{\partial}{\partial t} \bigg(\rho \big( e + \frac{1}{2}  \mathbf{u}\cdot\mathbf{u}\big)\bigg) + \nabla \bigg(\rho \mathbf{u} \big(\mathbf{h}+\frac{1}{2}\mathbf{u}\cdot\mathbf{u}\big) - \mathbf{u}\cdot \mathbf{\tau} + \mathbf{q}\bigg)= \rho \mathbf{f}\cdot\mathbf{u}+\rho r
    \end{equation}
    

Where *$\rho$* is the density, *p* is the pressure, * **u** * is the velocity, *$\tau$* is the viscous stress tensor, * **f** * is the body force vector, *e* is the internal energy, *h* is the enthalpy, * **q** * is the heat flux vector and *r* is a heat soure/sink term. 

Now the equation is written in terms of the conservative variable *$\rho$*, *$\mathbf{m} = \rho \mathbf{u}$*  , and *$e_{tot}$*   *$ = \rho \big( e + \frac{1}{2}  \mathbf{u}\cdot\mathbf{u}\big)$*

1. Conservation of mass equation

    \begin{equation}
    \label{MassConserv}
    \frac{\partial \rho}{\partial t} + \nabla \cdot \textbf{m}  = 0.
    \end{equation}

2. Conservation of momentum equation

    \begin{equation}
    \label{MomentumCons}
    \frac{\partial \mathbf{m}}{\partial t} + \nabla \cdot \mathbf{m} \frac{\mathbf{m}}{\rho} +\nabla (p\mathbf{I}-\mathbf{\tau}) = \rho \mathbf{f}
    \end{equation}
    
3. Conservation of energy equation

    \begin{equation}
    \label{EnergyCons}
    \frac{\partial e_{tot}}{\partial t} + \nabla \bigg((\rho+e_{tot})\frac{\mathbf{m}}{\rho} -\frac{\mathbf{m}}{\rho}\cdot\mathbf{\tau}+ \mathbf{q}\bigg)= \mathbf{f}\mathbf{m}+\rho r
    \end{equation}
    


It is possible to group the Navier-Stokes equations in a system with the help of the Einstein summation, considering $\mathbf{U} = (\rho, \mathbf{m}, e_{tot})^T$ as the vector of the conservative variables.

\begin{equation}
    \label{System}
    \frac{\partial (\mathbf{U})}{\partial t} + \frac{\partial \mathbf{F}_j (\mathbf{U})}{\partial x_j}+\frac{\partial \mathbf{G}_j (\mathbf{U})}{\partial x_j}-\mathbf{S}(\mathbf{U}) = \mathbf{0}, \quad in \quad \Omega \subset \mathbb{R}^d, t>0,
\end{equation}

\begin{equation}  
\label{Dirichlet}
 \textit{U}(\mathbf{U}_g)=\mathbf{U}_g,\quad \quad \quad \quad \quad \quad \quad \quad on\quad \Gamma_g,t>0,
 \end{equation}
 
 \begin{equation}
     \label{Neumann}
     \mathbf{F}_jn_j=\mathbf{h}, \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad on\quad \Gamma_n, t>0,
 \end{equation}
 
 \begin{equation}
     \label{InitialCond}
     \mathbf{U}=\mathbf{U}_0(\mathbf{x}), \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad in\quad \Omega, t=0
 \end{equation}
 
 The boundary conditions for the Dirichlet and Neumann boundaries are here introduced, together with the initial condition. 

In [48]:
 ### Construction of the variational equation
Ug = DefineVector('Ug',BlockSize)			# Dofs vector
H = DefineMatrix('H',BlockSize,dim)			# Gradient of U
f = DefineVector('f',dim)			        # Body force vector
rg = Symbol('rg', positive = True)		    # Source/Sink term
V = DefineVector('V',BlockSize)			    # Test function
Q = DefineMatrix('Q',BlockSize,dim)			# Gradient of V
acc = DefineVector('acc',BlockSize)         # Derivative of Dofs/Time
#G = DefineMatrix('G',BlockSize,dim)		# Diffusive Flux matrix
Gsc = DefineMatrix('G',BlockSize,dim)       # Diffusive Flux matrix with Shock Capturing


### 1.1 Convective Flux Matrix Implementation


The $\textit{(d+2)*d}$ $\textit{convective}$ flux matrix $\mathbf{F}$ is:

\begin{equation}
    \label{Fmat}
    \mathbf{F}_j(\mathbf{U}) = \bigg(m_j,\frac{m_j}{\rho}m_i+p\delta_{ij},(e_{tot}+p)\frac{m_j}{\rho}\bigg)^T \quad \quad 1\leq{i},j\leq{d}
\end{equation}

Expanded:

\begin{equation}
    \mathbf{F} =
  \begin{bmatrix}
    m_1 & m_2 & m_3 \\
    \frac{m_1}{\rho}m_1+p & \frac{m_1}{\rho}m_2 & \frac{m_1}{\rho}m_3\\
    \frac{m_2}{\rho}m_1+p & \frac{m_2}{\rho}m_2+p & \frac{m_2}{\rho}m_3\\
    \frac{m_2}{\rho}m_1+p & \frac{m_2}{\rho}m_2 & \frac{m_3}{\rho}m_3+p\\
    (e_{tot}+p)\frac{m_1}{\rho} & (e_{tot}+p)\frac{m_2}{\rho} & (e_{tot}+p)\frac{m_3}{\rho}
  \end{bmatrix}
\end{equation}


Where the pressure $\textit{p}$ is:

\begin{equation}
    \label{pressure}
    \textit{p} (\mathbf{U}) = (\gamma-1) \bigg(\textit{e}_{tot}-\frac{|\mathbf{m}|^2}{2 \rho} \bigg)
\end{equation}



From this

\begin{equation}
    \label{DiffAmat}
    \frac{\partial \mathbf{F}_j(\mathbf{U})}{\partial x_j}=\mathbf{A}_j(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}
\end{equation}

 it is possible to derive the $\textit{(d+2)*(d+2)*d}$ Euler Jacobian matrix $\mathbf{A}_j$ as: 

\begin{equation}
    \label{Amatr}
    \mathbf{A}_j(\mathbf{U}) = \frac{\partial\mathbf{F}_j(\mathbf{U})}{\partial \mathbf{U}}
\end{equation}


\begin{equation}
    \mathbf{A} =
  \begin{bmatrix}
    \frac{\partial m_j}{\partial \rho} & \frac{\partial m_j}{\partial \mathbf{m}} & \frac{\partial m_j}{\partial e_{tot}} \\
    \frac{\partial \big(\frac{m_j}{\rho}m_i+p\delta_{ij}\big)}{\partial\rho} & 
    \frac{\partial \big(\frac{m_j}{\rho}m_i+p\delta_{ij}\big)}{\partial\mathbf{m}}&
    \frac{\partial \big(\frac{m_j}{\rho}m_i+p\delta_{ij}\big)}{\partial e_{tot}}\\
    \frac{\partial \big((e_{tot}+p)\frac{m_j}{\rho}\big)}{\partial\rho} &
      \frac{\partial \big((e_{tot}+p)\frac{m_j}{\rho}\big)}{\partial\mathbf{m}} &
      \frac{\partial \big((e_{tot}+p)\frac{m_j}{\rho}\big)}{\partial e_{tot}}\\
  \end{bmatrix} \quad\quad\quad\quad\quad\quad 1\leq i, j\leq d
\end{equation}


For each direction d the matrix $\mathbf{a_d}$ is defined as below.

\begin{equation}
    \mathbf{a_1} =
  \begin{bmatrix}
   0 & 1 & 0 & 0 & 0 \\
    \frac{(\gamma -3)m_1^2}{2\rho^2}+\frac{(\gamma -1) }{2 \rho^2} (m_2^2+m_3^2)& 
    \frac{(3-\gamma)m_1}{\rho}& \frac{(1-\gamma)m_2}{\rho} & \frac{(1-\gamma)m_3}{\rho} & \gamma-1 \\
    \frac{-m_1 m_2}{\rho^2} & \frac{m_2}{\rho} & \frac{m_1}{\rho} & 0 & 0 \\
    \frac{-m_1 m_3}{\rho^2} & \frac{m_3}{\rho} & 0 & \frac{m_1}{\rho} & 0 \\
    -\gamma\frac{e_{tot}}{\rho^2}m_1+ (\gamma-1)\frac{m_1}{\rho}\big( \frac{m_1^2}{\rho^2}+ \frac{m_2^2}{\rho^2}+
    \frac{m_3^2}{\rho^2}\big) &
    \gamma \frac{e_{tot}}{\rho}+\frac{1-\gamma}{2\rho^2}\big(3m_1^2+m_2^2+m_3^2\big) & \frac{(1-\gamma)}{\rho^2}m_1m_2 &\frac{(1-\gamma)}{\rho^2}m_1m_3 & \frac{\gamma m_1}{\rho} \\
  \end{bmatrix} 
\end{equation}


\begin{equation}
    \mathbf{a_2} =
  \begin{bmatrix}
   0 & 0 & 1 & 0 & 0 \\
    -\frac{m_1m_2}{\rho^2}& \frac{m_2}{\rho}& \frac{m_1}{\rho} & 0 & 0 \\
    -\frac{m_2m_3}{\rho^2}& 0 & \frac{m_3}{\rho}& \frac{m_2}{\rho} & 0\\
    \frac{(\gamma -3)m_3^2}{2\rho^2}+\frac{(\gamma -1) }{2 \rho^2}(m_1^2+m_2^2) & \frac{1-\gamma}{\rho}m_1 & \frac{(1-\gamma)}{\rho}m_2 & \frac{(3-\gamma)}{\rho}m_3 & \gamma-1 \\
    -\gamma\frac{e_{tot}}{\rho^2}m_2+ (\gamma-1)\frac{m_2}{\rho}\big( \frac{m_1^2}{\rho^2}+ \frac{m_2^2}{\rho^2}+
    \frac{m_3^2}{\rho^2}\big) &
    \frac{(1-\gamma)}{\rho^2}m_1m_2 &  \gamma \frac{e_{tot}}{\rho}+\frac{1-\gamma}{2\rho^2}\big(m_1^2+3m_2^2+m_3^2\big)  &\frac{(1-\gamma)}{\rho^2}m_2m_3 & \frac{\gamma m_2}{\rho}\\
  \end{bmatrix} 
\end{equation}


\begin{equation}
    \mathbf{a_3} =
  \begin{bmatrix}
   0 & 0 & 0 & 1 & 0 \\
    -\frac{m_1m_3}{\rho^2}& \frac{m_3}{\rho}& 0 & \frac{m_1}{\rho} & 0 \\
    \frac{-m_2 m_3}{\rho^2} & 0 & \frac{m_3}{\rho} & \frac{m_2}{\rho} & 0 \\
    \frac{(\gamma -3)m_2^2}{2\rho^2}+\frac{(\gamma -1) }{2 \rho^2}(m_1^2+m_3^2) & \frac{1-\gamma}{\rho}m_1 & \frac{(3-\gamma)}{\rho}m_2 & \frac{(1-\gamma)}{\rho}m_3 & \gamma-1 \\
       -\gamma\frac{e_{tot}}{\rho^2}m_3+ (\gamma-1)\frac{m_3}{\rho}\big( \frac{m_1^2}{\rho^2}+ \frac{m_2^2}{\rho^2}+
    \frac{m_3^2}{\rho^2}\big) &
    \frac{(1-\gamma)}{\rho^2}m_1m_3 & \frac{(1-\gamma)}{\rho^2}m_2m_3 & \gamma \frac{e_{tot}}{\rho}+\frac{1-\gamma}{2\rho^2}\big(m_1^2+m_2^2+3m_3^2\big)  & \frac{\gamma m_3}{\rho}\\
  \end{bmatrix} 
\end{equation}


In [49]:
## Computation of the Convective Matrix
def computeA(dofs, params):
    print("\nCompute Convective Matrix \n")
    dim = params["dim"]				        # Spatial dimensions
    
    ## Unknown field definition
    F = DefineMatrix('F',dim+2,dim)		    # Convective Flux matrix 
    Ug = dofs                               # Data interpolation to the Gauss points
    
    ## Other symbols definitions
    y = params["gamma"]				        # Gamma (Cp/Cv)
   
    ## Pgauss - Pressure definition
    pg = (y-1)*Ug[dim+1]
    for i in range(0,dim):
        pg += (y-1)*(-Ug[i+1]*Ug[i+1]/(2*Ug[0]))
    
    ## F - Convective Flux Matrix definition
    for j in range(0,dim):
        F[0,j] = Ug[j+1]
    
    for i in range (1,dim+1):
        for j in range(0,dim):
            F[i,j] = Ug[i]*Ug[j+1]/Ug[0]
            if i==j+1:
               F[i,j]+=pg

    for j in range(0,dim):
        F[dim+1,j] = (Ug[dim+1]+pg)*Ug[j+1]/Ug[0]
 

    ## A - Jacobian Convective Matrix definition
    A = []

    for j in range(0,dim):
        tmp = DefineMatrix('tmp',dim+2,dim+2)
        for i in range(0,dim+2):
            for k in range(0,dim+2):
      	        tmp[i,k] = diff(F[i,j], Ug[k])
      	        #print(j,'	',i,k,'=',tmp[i,k]) 	   
        A.append(tmp)    
    return A   
 
## Printing the Convective Matrix
def printA(A,params):
    dim = params["dim"]
    tmp = []
    print("The convective matrix is:\n")
    for j in range(0,dim):
    	tmp = A[j]
    	for i in range(0,dim+2):
            for k in range(0,dim+2):
                print("A[",j,",",i,",",k,"]=",tmp[i,k],"\n")
       
    return 0

In [50]:
## Matrix Computation

A = computeA(Ug,params)
#printA(A,params)


Compute Convective Matrix 



### 1.3 Diffusive Flux Matrix Implementation

The diffusive matrix is related to the viscous and thermal properties of the fluid. The former are presented both in the conservation of momentum and energy, while the latter are necessary to define the conservation of energy only.

The shear stress tensor is defined as follow:

\begin{equation}
    \tau_{ij} (\mathbf{u}) = \mu \bigg(\frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i} \bigg) -\frac{2 \mu}{3} \bigg(\frac{\partial u_l}{\partial x_l} \bigg) \delta_{ij}
\end{equation}

with the dynamic viscosity $\mu$. The relation can be rewritten in terms of conservative variables as:

\begin{equation}
    \tau_{ij}(\mathbf{U}) = \frac{\mu}{\rho}\bigg(\frac{\partial m_i}{\partial x_j}+\frac{\partial m_j}{\partial x_i}\bigg)-\frac{2\mu}{3\rho}\bigg(\frac{\partial m_k}{\partial x_k}\bigg)\delta_{ij}-\frac{\mu}{\rho^2}\bigg(m_i \frac{\partial\rho}{\partial x_j}+m_j \frac{\partial \rho}{\partial x_i}\bigg)+\frac{2\mu}{3\rho^2}\bigg(m_k \frac{\partial \rho}{\partial x_k}\bigg)\delta_{ij} \quad i,j,k<{d}
\end{equation}

Substituting $\mathbf{H}(\mathbf{U})= \frac{\partial \mathbf{U}}{\partial x_j}$, which is a $\textit{(dim+2)*(dim)}$ matrix, and $\mathbf{U}$ in the equation:

\begin{equation}
    \tau_{ij}(\mathbf{U}) = \frac{\mu}{U_0}\bigg(H_{i+1,j}+H_{j+1,i}\bigg)-\frac{2\mu}{3U_0}\bigg(H_{k+1,k}\bigg)\delta_{ij}-\frac{\mu}{U_0^2}\bigg(U_{i+1} H_{0,j}+U_{j+1}H_{0,i}\bigg)+\frac{2\mu}{3U_0^2}\bigg(U_{k+1} H_{0,k}\bigg)\delta_{ij} \quad i,j,k<{d}
\end{equation}

The heat flux vector $q_i(\mathbf{U})$ is defined by the Fourier's law as:

\begin{equation}
    q_i(\theta) = -\lambda \frac{\partial \theta}{\partial x_i}
\end{equation}

where $\lambda$ is the thermal conductivity and $\theta$ is the temperature of the fluid.

Two additional equations are necessary in order to close the problem. Here the caloric equation and the perfect gas state equation are used in order to compute the pressure $\textit{p}$ and the speed of sound $\textit{c}$.

\begin{equation}
    e = c_v(\theta)\theta \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad p = \rho R \theta
\end{equation}

Where $\textit{R} = c_p - c_v$ and $\gamma = \frac{c_p}{c_v}$ is the ratio between the specific heat at constant pressure $c_p$ and the specific heat at constant volume $c_v$. The internal energy can be expressed in terms of the conservative variables leading to the following equation for the heat flux vector.

\begin{equation}
    e = \frac{e_{tot}}{\rho}- \frac{|\textbf{m}|^2}{2}
\end{equation}


\begin{equation}
    q_i(\mathbf{U}) = \frac{\lambda \textit{e}_{tot}}{\rho^2 \textit{c}_v}\frac{\partial \rho}{\partial x_i}-\frac{\lambda m_jm_j}{\rho^3 \textit{c}_v}\frac{\partial \rho}{\partial x_i}+\frac{\lambda m_j}{\rho^2 \textit{c}_v}\frac{\partial m_j}{\partial x_i}-\frac{\lambda}{\rho \textit{c}_v}\frac{\partial \textit{e}_{tot}}{\partial x_i}
\end{equation}

Which substituting $\mathbf{H}$ and $\mathbf{U}$ becomes:
\begin{equation}
    q_i(\mathbf{U}) = \frac{\lambda U_{d+1}}{U_0^2 \textit{c}_v}H_{0,i}-\frac{\lambda U_{j+1}U_{j+1}}{U_0^3 \textit{c}_v}H_{0,i}+\frac{\lambda U_{j+1}}{U_0^2 \textit{c}_v}H_{j+1,i}-\frac{\lambda}{U_0 \textit{c}_v}H_{d+1}\quad i,j<d
\end{equation} 

NB: $\rho$ is the first term of the vector of unknwons $\mathbf{U}$ and $\textit{e}_{tot}$ is the last one (Ug\[dim+1\]).
Remember to consider them as Unknowns and not fluid parameters.

The $\textit{(d+2)*d diffusive}$ flux matrix $\mathbf{G}$ is defined as follow.

\begin{equation}
    \label{Gmat}
    \mathbf{G}_j(\mathbf{U}) = \big(0,-\tau_{ji}, -\frac{m_i}{\rho}\tau_{ij}+q_j\big)^T \quad \quad i,j<{d}
\end{equation}

\begin{equation}
    \mathbf{G} =
  \begin{bmatrix}
    0 & 0 & 0 \\
    -\tau_{11} & -\tau_{21} & -\tau_{31}\\
    -\tau_{12} & -\tau_{22} & -\tau_{32}\\
    -\tau_{13} & -\tau_{23} & -\tau_{33}\\
    -\frac{m_1}{\rho}\tau_{11}-\frac{m_2}{\rho}\tau_{21}-\frac{m_3}{\rho}\tau_{31}+q_1 & 
   -\frac{m_1}{\rho}\tau_{12}-\frac{m_2}{\rho}\tau_{22}-\frac{m_3}{\rho}\tau_{32}+q_2 &
   -\frac{m_1}{\rho}\tau_{13}-\frac{m_2}{\rho}\tau_{23}-\frac{m_3}{\rho}\tau_{33}+q_3
  \end{bmatrix}
\end{equation}

The fourth order tensor $\mathbf{K}_{kj}(\mathbf{U})$ $\textit{(d+2)*(d+2)*d*d}$ can be manually derived in order to satisfy this expression:

\begin{equation}
    \label{K}
    \frac{\partial\mathbf{G}_j(\mathbf{U})}{\partial x_j} =- \frac{\partial}{\partial x_k}\bigg(\mathbf{K}_{kj}(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}\bigg)
\end{equation}

In this project the $\mathbf{K}$ tensor was not computed in order to simplify the implementation.
This does not affect the solution as long as using linear shape of functions, due to the fact that the tensor is a second order term.



In [51]:
## Computation of the Diffusive Matrix with Shock Capturing
def computeGsc(dofs,params,Hg,Gg,v_sc,k_sc):
    print("\nCompute Diffusive Matrix with Shock Capturing\n")
    dim = params["dim"]				                    # spatial dimensions
    
    ## Unknown fields definition
    H = Hg.copy()                               		# Gradient of U
    Gsc = Gg.copy()                               		# Diffusive Flux matrix 
    tau_stress = DefineMatrix('tau_stress',dim,dim)		# Shear stress tensor for Newtonian fluid
    q = DefineVector('q',dim)			                # Heat flux vector
    
    ## Other simbols definition
    c_v = params["c_v"]				                    # Specific Heat at Constant volume
    gamma = params["gamma"]				                # Gamma (Cp/Cv) 
    mu  = params["mu"]         			                # Dynamic viscosity 
    l = params["lambda"]			                    # Thermal Conductivity 
        
    ## Data interpolation to the Gauss points
    Ug = dofs             

    ## Pgauss - Pressure definition
    pg = Ug[dim+1]
    for i in range(0,dim):
        pg += (-Ug[i+1]*Ug[i+1]/(2*Ug[0]))
    pg *= (gamma-1)

    ## tau - Shear stress tensor definition
    for i in range(0,dim):
        for j in range(i,dim):
            if i!=j:
               tau_stress[i,j] = (mu/Ug[0])*(H[i+1,j]+H[j+1,i])-(mu/Ug[0]**2)*(Ug[i+1]*H[0,j]+Ug[j+1]*H[0,i])
            if i==j:
               tau_stress[i,j]= (2*mu/Ug[0])*H[i+1,i]-(2*mu/Ug[0]**2)*Ug[i+1]*H[0,i]
               for k in range(0,dim):
                   tau_stress[i,j]+= -(2*mu/(3*Ug[0]))*H[k+1,k]+(2*mu/(3*Ug[0]**2))*Ug[k+1]*H[0,k]
    
    for i in range(1,dim):
        for j in range(0,dim-1):
            if j!=i:
               tau_stress[i,j] = tau_stress[j,i]
               
    ## q - Heat flux vector definition
    for i in range(0,dim):
        q[i] = l*Ug[dim+1]/(Ug[0]**2*c_v)*H[0,i]-(l*H[dim+1,i])/(Ug[0]*c_v)
        for j in range(0,dim):
            q[i] += -l*Ug[j+1]**2/(c_v*Ug[0]**3)*H[0,i]+l/(Ug[0]**2*c_v)*Ug[j+1]*H[j+1,i] 
    #NB!!!There is an error in the definition of q[i] in the research proposal. 
    #The second term of the equation has an opposite sign!!!NB#
    ''' 
    G [(dim+2)*(dim)]
    
    0                                   0 
    -tau00                              -tau01
    -tau01                              -tau11
    -mu/rho*tau00-mv/rho*tau01+q0       -mu/rho*tau01-mv/rho*tau11+q1
    '''
    tau_sc = (1+(Ug[0]*v_sc)/mu)*tau_stress     # Stress tensor with shock capturing viscosity
    q_sc = (1+(Ug[0]*c_v*k_sc)/l)*q             # Heat flux with shock capturing conductivity
    
    ## Gsc - Diffusive Matrix definition 
    for j in range(0,dim):
        Gsc[0,j]= 0 			                #Mass equation related
       
    for i in range(1,dim+1):
        for j in range(0,dim):
            Gsc[i,j]=-tau_sc[i-1,j]		        #Moment equation related
    
    for j in range(0,dim):                      #Energy equation related
        Gsc[dim+1,j] = q_sc[j]
        for k in range(0,dim):
            Gsc[dim+1,j] += -Ug[k+1]*tau_sc[k,j]/Ug[0]

    return Gsc

## Printing the Diffusive Matrix G
def printG(G,params):
    dim = params["dim"]
    print("The diffusive matrix is:\n")
    for ll in range(dim+2):
        for mm in range(dim):
            print("G[",ll,",",mm,"]=",G[ll,mm],"\n")
       
    return 0

### 1.4 Shock Capturing technique

The diffusive term is modified in order to include additional artificial viscosity and conductivity. These terms are necessary to guarantee stability even when, due to shock waves, there are sharp gradients in the solution.
There are different methods to include diffusion to capture the nonlinearities generated by the physical phenomenum of shock. 
Below the implementation of an isotropic residual based shock capturing is presented.

In the isotropic case the artificial diffusion is added in all components, adding an extra term in the stress tensor and one in the heat flux vector.

\begin{equation}
    \breve{\tau_{ij}} = \bigg( 1+ \frac{\rho \nu_{sc}}{\mu}\bigg) \tau_{ij}
\end{equation}


\begin{equation}
    \breve{q_i} = \bigg( 1+ \frac{\rho c_v k_{sc}}{\lambda}\bigg) q_i
\end{equation}

where $\nu_{sc}$ and $\textit{k}_{sc}$ are respectively the kinematic viscosity and the thermal diffusivity coefficient related to the shock capturing.

The coefficients are computed using a residual based the technique. Below the definition of the residual related to the momenum equation and the one related to the total energy are shown. The nonlinear operator $\mathbf{L} (\mathbf{U} ; \mathbf{U})$ will be defined in details in section 2.1.

\begin{equation}
    \mathbf{R} (\mathbf{m_h}) = -\frac{\mathbf{U_m}}{\partial t}-\mathbf{L_m} (\mathbf{U};\mathbf{U_h} )
\end{equation}

\begin{equation}
    R(e_{tot}) =  -\frac{Ue_{tot}}{\partial t}-\mathbf{L}e_{tot} (\mathbf{U};\mathbf{U_h} )
\end{equation}

The coefficients are defined as follow:

\begin{equation}
    \nu_{sc} = \frac{1}{2} h \alpha \frac{\big| \mathbf{R}(\mathbf{m_h})\big|}{\big| \nabla \mathbf{m_h} \big |} \quad if \quad \big|\nabla \mathbf{m_h} \big|\neq 0,\quad \quad\quad \quad \quad \quad\quad\quad\quad\nu_{sc} = 0 \quad otherwise
\end{equation}

The Froebenius norm of the momentum gradient is computed as:

\begin{equation}
    \big| \nabla \mathbf{m_h} \big| = \Bigg(\sum_{i=1}^{d}\sum_{j=1}^{d} \bigg| \frac{\partial m_{i,h}}{\partial x_j}\bigg |^2 \Bigg) ^{\frac{1}{2}}
\end{equation}

\begin{equation}
    k_{sc} = \frac{1}{2} h \alpha \frac{\big| R(e_{tot},h) \big|}{\big | \nabla e_{tot},h \big|} \quad if \quad \big| \nabla e_{tot},h \big| \neq 0, \quad \quad\quad \quad \quad \quad\quad\quad k_{sc} = 0 \quad otherwise
\end{equation}

Nota bene: the shock capturing diffusion is activated in case the norm of the gradients is greater than a certain tolerance. The implementationi of the coefficients and the computation of the gradient norms is computed directly at the element level (in the .cpp template).

In [52]:
Gsc = computeGsc(Ug,params,H,Gsc,v_sc,k_sc)   
#printG(Gsc,params)


Compute Diffusive Matrix with Shock Capturing



### 1.3 Source Term Matrix Implementation

The $\textit{source}$ term vector $\mathbf{S(\mathbf{U})}$ is written as a product of a $\textit{(d+2)*(d+2)}$ reactive matrix $\mathbf{S}$ and the vector of unknowns.

\begin{equation}
    \label{S(U)}
    \mathbf{S}(\mathbf{U}) = (0,\rho\mathbf{f},\mathbf{f}\cdot\mathbf{m}+\rho r)^T
\end{equation}

\begin{equation}
    \label{S}
    \mathbf{S}=
    \begin{pmatrix} 0 & 0 & 0\\ \mathbf{f} & 0 & 0 \\ r &\mathbf{f}^T &0 \end{pmatrix}
\end{equation}

In [53]:
## Computation of the Source Matrix
def computeS(force,source,params):
    print("\nCompute Source Matrix \n")
    dim = params["dim"]				        # Spatial dimensions
    
    ## Unknown field definition	
    f = force.copy()					    # Body force vector
    r = source			 		            # Heat Source/Sink Term  #Symbol('r', positive = True) 
    '''
    S - Reactive Matrix definition     
     0  0  0   0
     fx 0  0   0
     fy 0  0   0
     r  fx fy  0
    '''
    S = zeros(dim+2,dim+2)                  # Reactive matrix (Source terms)

    
    for i in range(1,dim+1):
        S[i,0]   = f[i-1]
        S[dim+1,i] = f[i-1]        
    S[dim+1,0] = r
        
    return S

## Printing of the Source Matrix   
def printS(S,params):
    dim = params["dim"]				        # Spatial dimensions
    print("The source term matrix is:\n")
    for i in range (0,dim+2):
        for j in range (0,dim+2):
            print("S[",i,",",j,"]=",S[i,j],"\n")

    return 0

In [54]:
S = computeS(f,rg,params)
#printS(S,params)


Compute Source Matrix 



## 2. Approximation of the subscales 

Nota bene: the diffusive term is neglected in the approximation of the subscales (in the definition of the non-linear operator and its adjoint) to simplify the problem.
As explained previously this will not affect the solution as the second order terms related with the diffusion cannot nevertheless be detected by the linear elements used here. 

The finite element space is subdivided into a coarse and a fine subgrid space.<br>
 <center>$\mathbf{V} = \mathbf{V}_h+\widetilde{\mathbf{V}}, \quad \mathbf{V}_h \subset \textit{W}_h$</center>                    
In the same way, the $\mathbf{U}$ vector is decomposed in: <br>
<center>$ \mathbf{U} = \mathbf{U}_h + \widetilde{\mathbf{U}}, \quad \mathbf{U} \in \textit{W}$<center><br>

In order to understand the final formulation of the variational problem we need to define the nonlinear operator $\mathbf{\textit{L}}(\mathbf{U};\mathbf{U})$ , its adjoint $\mathbf{\textit{L}^{*}}(\mathbf{U};\mathbf{V}_h)$ and the finite element residual $\mathbf{R}(\mathbf{U};\mathbf{U}_h)$, together with the stabilization matrix $\mathbf{\tau}$.



### 2.1 Nonlinear Operator Definition

The problem can be rewritten using  the nonlinear operator $\mathbf{\textit{L}}(\mathbf{U};\mathbf{U})$ as below.


\begin{equation}
    \frac{\partial \mathbf{U}}{\partial t} + \mathbf{\textit{L}}(\mathbf{U} ; \mathbf{U}) = \mathbf{0}, \quad in \quad \Omega \subset \mathbb{R}^d, t>0,
\end{equation}

\begin{equation}  
 \textit{U}(\mathbf{U}_g)=\mathbf{U}_g,\quad \quad \quad \quad \quad \quad \quad \quad on\quad \Gamma_g,t>0,
 \end{equation}
 
 \begin{equation}
     \mathbf{F}_jn_j=\mathbf{h}, \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad on\quad \Gamma_n, t>0,
 \end{equation}
 
 \begin{equation}
     \mathbf{U}=\mathbf{U}_0(\mathbf{x}), \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad in\quad \Omega, t=0
 \end{equation}
Where $\mathbf{\textit{L}}$ is define as here.
  
\begin{equation}
    \mathbf{\textit{L}}(\mathbf{U};\mathbf{U}) = \mathbf{A}_j\frac{\partial \mathbf{U}}{\partial x_j}-\mathbf{S} \mathbf{U}
\end{equation}




In [55]:
## Nonlinear operator definition   
l1 = Matrix(zeros(dim+2,1))		            # Convective Matrix*Gradient of U
A_small = []
for j in range(0,dim):
    A_small = A[j]
    for ll in range(BlockSize):
        for mm in range(BlockSize):
            l1[ll] += A_small[ll,mm]*H[mm,j]

l3 = S*Ug				                    # Source term
print("\nCompute Non-linear operator\n")
L = l1-l3                                   # Nonlinear operator



Compute Non-linear operator



### 2.1 Nonlinear Adjoint Operator Definition
The nonlinear adjoint operator $\mathbf{\textit{L}^{*}}$ is here applied to the test functions vector $\mathbf{V}_h$.


\begin{equation}
    \label{Nonlinoper}
    \mathbf{\textit{L}^{*}}(\mathbf{U};\mathbf{V}_h) =-\frac{\partial}{\partial x_j}\bigg(\mathbf{A}_j^{T}(\mathbf{U})\mathbf{V}_h\bigg) - \mathbf{S}^T \mathbf{V}_h
\end{equation}

In [56]:
## Nonlinear adjoint operator definition  
m1 = Matrix(zeros(dim+2,1))		            # Convective term
psi = Matrix(zeros(dim+2,dim))

for j in range(0,dim):
    A_T = A[j].transpose()
    for l in range(0,dim+2):
        for m in range(0,dim+2):
            psi[l,j] += A_T[l,m]*Q[m,j]                 
            for n in range(0,dim+2):
                psi[l,j] +=diff(A_T[l,m],Ug[n])*H[n,j]*V[m]   

for s in range(0,dim+2):
    for j in range(0,dim):
        m1[s] += psi[s,j]

m3 = S.transpose()*V			            # Source term

print("\nCompute Non-linear adjoint operator\n")
L_adj = -m1-m3                              # Nonlinear adjoint operator


Compute Non-linear adjoint operator



### 2.3 Residual of the Finite Element Scale

The finite element residual is:
\begin{equation}
    \label{R}
    \mathbf{R}(\mathbf{U};\mathbf{U_h}) = -\frac{\partial \mathbf{U}_h}{\partial t}-\mathbf{\textit{L}}(\mathbf{U};\mathbf{U}_h)
\end{equation}

In [57]:
## Residual definition     
res = -acc - L

Below the definition of the momentum and energy residual, as presented in paragraph 1.4.

In [58]:
## Istotropic Residual Based Shock Capturing
res_m = Matrix(zeros(dim,1))                # Momentum residual
for i in range(0,dim):
    res_m[i,0] = res[i+1,0]

res_e = Matrix(zeros(1,1))                  # Total Energy residual
res_e[0,0] = res[dim+1]

### 2.4 The Stabilization matrix 

The stabilization matrix $\mathbf{\tau}$ is a $\textit{(d+2)*(d+2)}$ diagonal matrix defined like this:<br>
<br>
\begin{equation}
    \label{tau}
    \boldsymbol{\tau}^{-1} = diag \big(\tau^{-1}_1,\tau^{-1}_2\mathbf{I}_d,\tau^{-1}_3 \big)  \\
    \tau^{-1}_1 = \textit{c}_2 \frac{\lvert {\frac{\mathbf{m}} {\rho}}\rvert +c}{\textit{h}} \\
    \tau^{-1}_2 = \textit{c}_1 \frac{\nu}{\textit{h}^2} +\textit{c}_2 \frac{\lvert {\frac{\mathbf{m}} {\rho}}\rvert +c}{\textit{h}} \\
    \tau^{-1}_3 = \textit{c}_1 \frac{\lambda}{\rho \textit{c}_p \textit{h}^2} +\textit{c}_2 \frac{\lvert {\frac{\mathbf{m}} {\rho}}\rvert +c}{\textit{h}} \\
\end{equation}

Where $\textit{c}$ is the speed of sound, while $\textit{c}_1$ and $\textit{c}_2$ are algorithm constants (here $\textit{c}_1$ = 4 and $\textit{c}_2$ = 2)


Nota bene: the definition of the tau terms is done directly at the element level (in the *.cpp file)

In [59]:
def computeTau(params):
    print("\nCompute Stabilization Matrix\n")
    dim = params["dim"]				# Spatial dimensions
    Tau = zeros(dim+2,dim+2)        # Stabilization Matrix
    
    tau1 = Symbol('tau1')
    tau2 = Symbol('tau2')
    tau3 = Symbol('tau3')
 
    Tau[0,0] = tau1
    for i in range (0,dim):
        Tau[i+1,i+1] = tau2
    Tau[dim+1,dim+1] = tau3
    return(Tau)



def printTau(Tau, params):
    dim = params["dim"]				# Spatial dimensions
    print("The Stabilization term matrix is:\n")
    for i in range (0,dim+2):
        for j in range (0,dim+2):
            print("Tau[",i,",",j,"]=",Tau[i,j],"\n")

    return 0

In [60]:
Tau = computeTau(params)
printTau(Tau,params)


Compute Stabilization Matrix

The Stabilization term matrix is:

Tau[ 0 , 0 ]= tau1 

Tau[ 0 , 1 ]= 0 

Tau[ 0 , 2 ]= 0 

Tau[ 0 , 3 ]= 0 

Tau[ 0 , 4 ]= 0 

Tau[ 1 , 0 ]= 0 

Tau[ 1 , 1 ]= tau2 

Tau[ 1 , 2 ]= 0 

Tau[ 1 , 3 ]= 0 

Tau[ 1 , 4 ]= 0 

Tau[ 2 , 0 ]= 0 

Tau[ 2 , 1 ]= 0 

Tau[ 2 , 2 ]= tau2 

Tau[ 2 , 3 ]= 0 

Tau[ 2 , 4 ]= 0 

Tau[ 3 , 0 ]= 0 

Tau[ 3 , 1 ]= 0 

Tau[ 3 , 2 ]= 0 

Tau[ 3 , 3 ]= tau2 

Tau[ 3 , 4 ]= 0 

Tau[ 4 , 0 ]= 0 

Tau[ 4 , 1 ]= 0 

Tau[ 4 , 2 ]= 0 

Tau[ 4 , 3 ]= 0 

Tau[ 4 , 4 ]= tau3 



0

## 3. Variational Formulation

Finally the problem can be described using the equation below.  
 
\begin{equation}
    \label{final}
    \bigg(\mathbf{V}_h,\frac{\partial \mathbf{U}_h}{\partial t}\bigg) + \bigg(\mathbf{V}_h,\mathbf{A}_j(\mathbf{U}_h)\frac{\partial \mathbf{U}_h}{\partial x_j}\bigg) - \bigg(\frac{\partial \mathbf{V}_h}{\partial x_j},\mathbf{G}_{j}(\mathbf{U}_h)\bigg) - \bigg(\mathbf{V}_h,\mathbf{S} \mathbf{U}_h\bigg) + \sum_{K} \bigg\langle \mathbf{\textit{L}^{*}}(\mathbf{U}_h;\mathbf{V}_h),
    \boldsymbol{\tau} (\mathbf{U}_h) \mathbf{R}(\mathbf{U}_h) \bigg\rangle_K = 0 \quad \forall \mathbf{V}_h \in  \textit{W}
\end{equation}

In [61]:
## Variational Formulation - Final equation
n1 = V.transpose()*acc		                # Mass term - FE scale

temp = zeros(dim+2,1)
A_smalll = []
for i in range(0,dim):
    A_smalll = A[i]
    for ll in range(BlockSize):
        for mm in range(BlockSize):
            temp[ll] += A_smalll[ll,mm]*H[mm,i]

n2 = V.transpose()*temp			            # Convective term - FE scale

n3 = Matrix(zeros(1,1))                     # Diffusive term - FE scale

for j in range(0,dim):
    for k in range(BlockSize):
        n3[0,0] += Q[k,j]*(-Gsc[k,j])       # G with shock capturing - FE scale

n4 = -V.transpose()*(S*Ug)		            # Source term - FE scale

n5 = L_adj.transpose()*(Tau*res)	        # VMS_adjoint - Subscales 

print("\nCompute Variational Formulation\n")
rv = n1+n2+n3+n4+n5 			            # VARIATIONAL FORMULATION - FINAL EQUATION



Compute Variational Formulation



### 3.1 Numerical discretization of the Variational Formulation

In [62]:
### Substitution of the discretized values at the gauss points
print("\nSubstitution of the discretized values at the gauss points\n")

## Data interpolation at the gauss points
U_gauss = U.transpose()*N
w_gauss = w.transpose()*N
f_gauss = f_ext.transpose()*N                     #COMMENT for manufactured solution
acc_gauss = (bdf0*U+bdf1*Un+bdf2*Unn).transpose()*N
r_gauss = (r.transpose()*N)[0]                    #COMMENT for manufactured solution  
#r_gauss = Symbol('r_gauss', positive = True)     #USED for manufactured solution

## Gradients computation
grad_U = DfjDxi(DN,U).transpose()
grad_w = DfjDxi(DN,w).transpose()

print("\nSubstitution in the variational formulation\n")
SubstituteMatrixValue(rv, Ug, U_gauss)
SubstituteMatrixValue(rv, acc, acc_gauss)
SubstituteMatrixValue(rv, H, grad_U)
SubstituteMatrixValue(rv, V, w_gauss)
SubstituteMatrixValue(rv, Q, grad_w)
SubstituteMatrixValue(rv, f, f_gauss)           #COMMENT for manufactured solution
SubstituteScalarValue(rv, rg, r_gauss)          #COMMENT for manufactured solution

print("\nSubstitution in the residual of momentum\n")
SubstituteMatrixValue(res_m, Ug, U_gauss)
SubstituteMatrixValue(res_m, acc, acc_gauss)
SubstituteMatrixValue(res_m, H, grad_U)
SubstituteMatrixValue(res_m, f, f_gauss)       #COMMENT for manufactured solution
SubstituteScalarValue(res_m, rg, r_gauss)      #COMMENT for manufactured solution

print("\nSubstitution in the residual of total energy\n")
SubstituteMatrixValue(res_e, Ug, U_gauss)
SubstituteMatrixValue(res_e, acc, acc_gauss)
SubstituteMatrixValue(res_e, H, grad_U)
SubstituteMatrixValue(res_e, f, f_gauss)       #COMMENT for manufactured solution
SubstituteScalarValue(res_e, rg, r_gauss)      #COMMENT for manufactured solution




Substitution of the discretized values at the gauss points


Substitution in the variational formulation


Substitution in the residual of momentum


Substitution in the residual of total energy



Matrix([[-N_0*(U_0_4*bdf0 + Un_0_4*bdf1 + Unn_0_4*bdf2) - N_1*(U_1_4*bdf0 + Un_1_4*bdf1 + Unn_1_4*bdf2) - N_2*(U_2_4*bdf0 + Un_2_4*bdf1 + Unn_2_4*bdf2) - N_3*(U_3_4*bdf0 + Un_3_4*bdf1 + Unn_3_4*bdf2) - gamma*(DN_0_0*U_0_4 + DN_1_0*U_1_4 + DN_2_0*U_2_4 + DN_3_0*U_3_4)*(N_0*U_0_1 + N_1*U_1_1 + N_2*U_2_1 + N_3*U_3_1)/(N_0*U_0_0 + N_1*U_1_0 + N_2*U_2_0 + N_3*U_3_0) - gamma*(DN_0_1*U_0_4 + DN_1_1*U_1_4 + DN_2_1*U_2_4 + DN_3_1*U_3_4)*(N_0*U_0_2 + N_1*U_1_2 + N_2*U_2_2 + N_3*U_3_2)/(N_0*U_0_0 + N_1*U_1_0 + N_2*U_2_0 + N_3*U_3_0) - gamma*(DN_0_2*U_0_4 + DN_1_2*U_1_4 + DN_2_2*U_2_4 + DN_3_2*U_3_4)*(N_0*U_0_3 + N_1*U_1_3 + N_2*U_2_3 + N_3*U_3_3)/(N_0*U_0_0 + N_1*U_1_0 + N_2*U_2_0 + N_3*U_3_0) + (gamma - 1)*(DN_0_0*U_0_2 + DN_1_0*U_1_2 + DN_2_0*U_2_2 + DN_3_0*U_3_2)*(N_0*U_0_1 + N_1*U_1_1 + N_2*U_2_1 + N_3*U_3_1)*(N_0*U_0_2 + N_1*U_1_2 + N_2*U_2_2 + N_3*U_3_2)/(N_0*U_0_0 + N_1*U_1_0 + N_2*U_2_0 + N_3*U_3_0)**2 + (gamma - 1)*(DN_0_0*U_0_3 + DN_1_0*U_1_3 + DN_2_0*U_2_3 + DN_3_0*U_3_3)*(N_0*U_0_1 + 

### 3.2 Computation of the RHS vector and LHS matrix


In [63]:
dofs = Matrix(zeros(nnodes*(dim+2),1))
testfunc = Matrix(zeros(nnodes*(dim+2),1))
for i in range(0,nnodes):
     for j in range(0,dim+2):
        dofs[i*(dim+2)+j] = U[i,j]
        testfunc[i*(dim+2)+j] = w[i,j]

## Compute LHS and RHS
print("\nCompute RHS\n")
rhs = Compute_RHS(rv.copy(), testfunc, do_simplifications)
rhs_out = OutputVector_CollectingFactors(rhs, "rhs", mode)

print("\nCompute LHS\n")
lhs = Compute_LHS(rhs, testfunc, dofs, do_simplifications) # Compute the LHS
lhs_out = OutputMatrix_CollectingFactors(lhs, "lhs", mode)


Compute RHS


Compute LHS



In [64]:
## Residual for shock capturing
res_m_out = OutputMatrix_CollectingFactors(res_m, "res_m", mode)
res_e_out = OutputMatrix_CollectingFactors(res_e, "res_e", mode)


### 3.3 Writing output to *.cpp file

In [65]:
## Reading Template File
print("\nReading compressible_navier_stokes_cpp_template.cpp\n")
if(dim==2):
        templatefile = open("compressible_navier_stokes_cpp_template2D.cpp")
        outstring=templatefile.read()
        outstring = outstring.replace("//substitute_lhs_2D", lhs_out)
        outstring = outstring.replace("//substitute_rhs_2D", rhs_out)
        outstring = outstring.replace("//substitute_res_m_2D", res_m_out)
        outstring = outstring.replace("//substitute_res_e_2D", res_e_out)
        ## Write the modified template
        print("\nWriting compressible_navier_stokes2D.cpp\n")
        out = open("../../custom_elements/compressible_navier_stokes2D.cpp",'w')
        out.write(outstring)
        out.close()
elif(dim == 3):
        templatefile = open("compressible_navier_stokes_cpp_template3D.cpp")
        outstring=templatefile.read()
        outstring = outstring.replace("//substitute_lhs_3D", lhs_out)
        outstring = outstring.replace("//substitute_rhs_3D", rhs_out)
        outstring = outstring.replace("//substitute_res_m_3D", res_m_out)
        outstring = outstring.replace("//substitute_res_e_3D", res_e_out)

        ## Write the modified template
        print("\nWriting compressible_navier_stokes3D.cpp\n")
        out = open("../../custom_elements/compressible_navier_stokes3D.cpp",'w')
        out.write(outstring)
        out.close()
print("\nCompressible Navier Stokes Element Generated\n")


Reading compressible_navier_stokes_cpp_template.cpp


Writing compressible_navier_stokes3D.cpp


Compressible Navier Stokes Element Generated

